In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./dataset/recipes.csv")

In [44]:
data.isna().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
DatePublished                      0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                       17237
RecipeIngredientQuantities         3
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeYield                   348071
R

In [6]:
data_filtrada = data.drop(columns=["AggregatedRating", "ReviewCount", "RecipeYield", "CookTime"])
data_filtrada = data_filtrada.dropna()

In [7]:
data_filtrada.to_csv('./dataset/recipes_sin_valores_nulos.csv')

In [8]:
data_filtrada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327965 entries, 0 to 522514
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RecipeId                    327965 non-null  int64  
 1   Name                        327965 non-null  object 
 2   AuthorId                    327965 non-null  int64  
 3   AuthorName                  327965 non-null  object 
 4   PrepTime                    327965 non-null  object 
 5   TotalTime                   327965 non-null  object 
 6   DatePublished               327965 non-null  object 
 7   Description                 327965 non-null  object 
 8   Images                      327965 non-null  object 
 9   RecipeCategory              327965 non-null  object 
 10  Keywords                    327965 non-null  object 
 11  RecipeIngredientQuantities  327965 non-null  object 
 12  RecipeIngredientParts       327965 non-null  object 
 13  Calories           

In [ ]:
data_procesada = data_filtrada.drop(columns=["AuthorId", "AuthorName", "DatePublished", "PrepTime",
                   "Images", "Name", "RecipeInstructions", "Description"])

In [ ]:
data_procesada.head()

,RecipeId,TotalTime,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
0,38,PT24H45M,Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0
1,39,PT4H25M,Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0
2,40,PT35M,Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0
3,41,PT24H20M,Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0
4,42,PT50M,Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0


In [ ]:
def eliminar_sobrante(row): # elimina la c y los parentesis que roodean el texto en algunas columnas y las ocmillas sobrantes
    columnas = ["Keywords", "RecipeIngredientQuantities", "RecipeIngredientParts"]
    for nombre in columnas:
        row[nombre] = row[nombre][2:-1].replace('"', '').split(', ')
    
    return row

data_procesada = data_procesada.apply(eliminar_sobrante, axis=1)

In [ ]:
data_procesada.head()

,RecipeId,TotalTime,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
0,38,PT24H45M,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0
1,39,PT4H25M,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1...","[saffron, milk, hot green chili peppers, onion...",1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0
2,40,PT35M,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1/2, 1, NA, 1 1/2, NA, 3/4]","[sugar, lemons, rind of, lemon, zest of, fresh...",311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0
3,41,PT24H20M,Soy/Tofu,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/...","[extra firm tofu, eggplant, zucchini, mushroom...",536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0
4,42,PT50M,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0


In [ ]:
data_procesada.to_csv('./dataset/recipes_procesado.csv')

In [33]:
data_procesada = pd.read_csv('./dataset/recipes_procesado.csv')

In [34]:
data_procesada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327965 entries, 0 to 327964
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  327965 non-null  int64  
 1   RecipeId                    327965 non-null  int64  
 2   TotalTime                   327965 non-null  object 
 3   RecipeCategory              327965 non-null  object 
 4   Keywords                    327965 non-null  object 
 5   RecipeIngredientQuantities  327965 non-null  object 
 6   RecipeIngredientParts       327965 non-null  object 
 7   Calories                    327965 non-null  float64
 8   FatContent                  327965 non-null  float64
 9   SaturatedFatContent         327965 non-null  float64
 10  CholesterolContent          327965 non-null  float64
 11  SodiumContent               327965 non-null  float64
 12  CarbohydrateContent         327965 non-null  float64
 13  FiberContent  

In [ ]:
data_entrenamiento = pd.read_csv("./dataset/recipes_entrenamiento.csv")

hola


0     c("blueberries", "granulated sugar", "vanilla ...
1     c("saffron", "milk", "hot green chili peppers"...
2     c("sugar", "lemons, rind of", "lemon, zest of"...
3     c("extra firm tofu", "eggplant", "zucchini", "...
4     c("plain tomato juice", "cabbage", "onion", "c...
5     c("graham cracker crumbs", "sugar", "butter", ...
6     c("chicken", "butter", "flour", "milk", "celer...
7     c("sugar", "margarine", "egg", "flour", "salt"...
8                             c("rice vinegar", "haeo")
9     c("butter", "brown sugar", "granulated sugar",...
10    c("margarine", "cake flour", "baking powder", ...
11    c("fresh mushrooms", "butter", "boneless skinl...
12    c("flour", "sugar", "baking powder", "salt", "...
13    c("water", "black tea", "cinnamon stick", "mil...
14    c("instant coffee", "sugar", "nonfat dry milk ...
15    c("all-purpose flour", "sugar", "baking powder...
16    c("carrots", "eggs", "white sugar", "all-purpo...
17    c("avocados", "tomatoes", "salt", "garlic"